In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.by import By

In [2]:
import pymongo
client = pymongo.MongoClient("localhost", 27017)
db = client["mangoplate_db"]
collection = db["hongdae_lv"] 

In [3]:
import requests

headers = {
    'authority': 'stage.mangoplate.com',
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.9',
    'content-type': 'application/x-www-form-urlencoded',
    'origin': 'https://www.mangoplate.com',
    'referer': 'https://www.mangoplate.com/',
    'sec-ch-ua': '"Not/A)Brand";v="99", "Google Chrome";v="115", "Chromium";v="115"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
}

In [4]:
data = {
    'language': 'eng',
    'device_uuid': 'LWP4b16912155737270863wHgiQ',
    'device_type': 'web',
    'start_index': '0',
    'request_count': '20',
    'keyword': 'Hongdae',
    'filter': '{"subcuisine_codes":[],"metro_codes":[],"price_codes":[],"cuisine_codes":[],"is_parking_available":0}',
    'order_by': '2',
}

In [5]:
# response = requests.post('https://stage.mangoplate.com/api/v5/search/by_keyword.json', headers=headers, data=data)
# print(response.text)

In [6]:
all_data = []

In [7]:
counter = 0  # Initialize the counter

while counter < 200:  # Continue until the counter reaches 200
    response = requests.post('https://stage.mangoplate.com/api/v5/search/by_keyword.json', headers=headers, data=data)
    
    if response.status_code == 200:
        result_data = response.json()
        restaurants = result_data.get('result', [])
        
        if not restaurants:  # Check if there is no data (empty list)
            break   # No more data available, break out of the loop

        for restaurant_info in restaurants:
            # Get the restaurant data
            restaurant_data = restaurant_info.get('restaurant', {})

            # Insert the restaurant data into MongoDB
            collection.insert_one(restaurant_data)
            counter += 1
            if counter >= 200:  # Check if the counter reaches 200
                break
            
        all_data.extend(restaurant_info.get('restaurant', {}) for restaurant_info in restaurants)

        # Update the start_index for the next page
        data['start_index'] = str(int(data['start_index']) + int(data['request_count']))
    else:
        print(f"Request failed with status code: {response.status_code}")
        break

In [8]:
all_data[0]

{'restaurant_uuid': 438965,
 'cusine_code': 3,
 'subcusine_code': 19,
 'region_code': 2,
 'metro_code': 12,
 'price_range_code': 1,
 'latitude': '37.5482541',
 'longitude': '126.921099',
 'status_code': 1,
 'phone_number': '',
 'phone1': '',
 'is_verified': 1,
 'name': 'Mentakamusyo',
 'branch_name': '',
 'address': '316-3, Sangsu-dong, Mapo-gu, Seoul',
 'road_address': '316-3, Sangsu-dong, Mapo-gu, Seoul',
 'building_name': '',
 'floor': '',
 'restaurant_key': 'V1pDpcDIntci',
 'web_url': 'https://www.mangoplate.com/restaurants/V1pDpcDIntci',
 'review_count': 40,
 'comment_count': 40,
 'picture_count': 240,
 'view_count': 63072,
 'beenhere_count': 40,
 'wannago_count': 1784,
 'checkin_count': 91,
 'rating': 4.8,
 'official_rating_available': 1,
 'full_address': '316-3, Sangsu-dong, Mapo-gu, Seoul',
 'full_road_address': '316-3, Sangsu-dong, Mapo-gu, Seoul',
 'full_origin_address': '',
 'full_origin_road_address': '',
 'picture_url': 'https://mp-seoul-image-production-s3.mangoplate.com/

In [9]:
len(all_data)

200

In [10]:
def get_restaurant_data(url):
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    fields = ['Address', 'Phone Number', 'Cuisine', 'Price Range', 'Parking', 'Business Hours', 'Break Time', 'Last order', 'Day Off']

    data_dict = {}
    
    details = driver.find_elements(By.XPATH, "/html/body/main/article/div[1]/div[1]/div/section[1]/table")
    data_list = [detail.text for detail in details]

    # Parse the lines in data_list to extract fields and values and create a dictionary
    lines = data_list[0].split('\n')
    current_field = None
    current_value = ""
    for line in lines:
        line = line.strip()
        if line and all(field not in line for field in ['Website']):
            if line != "Menu":
                if any(field in line for field in fields):
                    if current_field:
                        data_dict[current_field] = current_value.strip()
                    
                    for field in fields:
                        if field in line:
                            current_field = field
                            break
                    
                    current_value = line.replace(current_field, "", 1).strip()
                else:
                    current_value += " " + line
    
    data_dict[current_field] = current_value.strip()
    
    try:
        website = driver.find_element(By.CSS_SELECTOR, ".under_line").get_attribute("href")
    except Exception as e:
        website = ""
    try:
        favorite = driver.find_element(By.XPATH, "/html/body/main/article/div[1]/div[1]/div/section[1]/header/div[2]/span[3]").text.replace("\n", "").strip()
    except Exception as e:
        favorite = ''
    
    data_dict['website'] = website
    return data_dict

    driver.quit()

In [ ]:
import pymongo

if __name__ == "__main__":
    result = collection.find({}, {"web_url": 1, "_id":0})
    result = list(result)
    pv_urls = [(item['web_url']).replace('/restaurants/', '/en/restaurants/') for item in result]

    # Define a list to store scraped data
    scraped_data_list = []

    # Scrape data for each URL and store it in the 'scraped_data_list'
    for url in pv_urls:    
        scraped_data = get_restaurant_data(url)
        print(f"Scraped data for {url}: {scraped_data}")

        if scraped_data is not None and isinstance(scraped_data, dict):
            scraped_data["restaurant_url"] = url  # Add the 'restaurant_url' key to the scraped data
            scraped_data_list.append(scraped_data)
        else:
            print(f"Error: Scraped data is not a dictionary for {url}")

    # Upsert the data into MongoDB:
    try:
        for data in scraped_data_list:
            pv_collection = "hongdae_pv"
            pvcollection = db[pv_collection]
            url = data["restaurant_url"]  # Fetch the URL from the correct location in the data dictionary
            
            pvcollection.update_one({"restaurant_url": url}, {"$set": data}, upsert=True)
        print("Data upserted into MongoDB successfully.")
    except Exception as e:
        print("Error while upserting data into MongoDB:", e)

Scraped data for https://www.mangoplate.com/en/restaurants/V1pDpcDIntci: {'Address': '316-3, Sangsu-dong, Mapo-gu, Seoul 서울특별시 마포구 와우산로13길 49-10', 'Cuisine': 'Ramen, Soba, Udon', 'Price Range': 'Less than 10,000 Won', 'Parking': 'No Parking Space', 'Business Hours': '11:20 - 20:00', 'Break Time': '15:00 - 17:30', 'Last order': '14:30, 20:00', 'website': 'http://www.instagram.com/mentaka_musyo'}
Scraped data for https://www.mangoplate.com/en/restaurants/HHxp24PCqgZn: {'Address': '465-1, Seogyo-dong, Mapo-gu, Seoul 1F 102호 서울특별시 마포구 동교로 126 1F 102호', 'Phone Number': '070-8801-2053', 'Cuisine': 'Japanese Cutlet', 'Price Range': '10,000 ~ 20,000 Won', 'Parking': 'No Parking Space', 'Business Hours': '11:30 - 21:00', 'Break Time': '15:00 - 17:30', 'Last order': '14:30, 20:30', 'Day Off': 'Sun + 1', 'website': 'http://www.instagram.com/_kawakatsu_/'}
Scraped data for https://www.mangoplate.com/en/restaurants/-SvQDki5HNzQ: {'Address': '403-23, Seogyo-dong, Mapo-gu, Seoul 서울특별시 마포구 독막로9길 26', 

In [ ]:
# Close the MongoDB client
client.close()